In [32]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [47]:
## change ##
batch_size = 16
clip = .5

Prepare data

In [4]:
train_gen = SequenceGenSpec(train,Is2D=False)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False,Is2D=False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [48]:
convNets = Sequential(Dropout(0.5),
                      ConvBatchLeaky(1,4,(5,3),stride=(2,1)),
                      Dropout(0.5),
                      ConvBatchLeaky(4,8,(5,3),stride=(2,1)),
                      Dropout(0.5),
                      ConvBatchLeaky(8,16,(5,5),stride=(2,2)),
                      Dropout(0.5),
                      ConvBatchLeaky(16,32,(3,3),stride=(2,2)),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.view(-1,32*6,192)),
                      GRU_NCL(192,192,2,batch_first=True,dropout=0.5,bidirectional=True,returnH=True))

In [49]:
linear = Sequential(Dropout(0.5),Linear(192*4,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [50]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [51]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [52]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [53]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.247628405690193, val_loss:3.106119394302368
epoch:1, train_loss:2.6171988936570973, val_loss:2.529010057449341
epoch:2, train_loss:2.3487037947544684, val_loss:2.401848793029785
epoch:3, train_loss:2.2711781280545087, val_loss:2.468982458114624
epoch:4, train_loss:2.2574153800423327, val_loss:2.4400408267974854
Training completed in 46.57829761505127s


Submission

In [54]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [55]:
test_gen = SequenceGenSpecTest(submission.seg_id.tolist(),Is2D=False)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [ ]:
submission.iloc[:,1] = predict(model,test_gen)

In [ ]:
submission.to_csv('../Submission/sub_spec_3d.csv',index=False)